In [ ]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops

class SmoothingLayer(MessagePassing):
    def __init__(self):
        super(SmoothingLayer, self).__init__(aggr='add')  # Use "add" aggregation.

    def forward(self, x, edge_index):
        # Pass messages between nodes.
        edge_index,_ = add_self_loops(edge_index)
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        # Compute the L1 norm of the input features for each edge of the graph.
        # out =  (torch.sum(torch.pow(x_i - x_j,2),dim=1,keepdim=True))
        out = torch.linalg.vector_norm(x_i - x_j, ord=2, dim=1,keepdim=True)
        return out
   

    def update(self, aggr_out):
        # Return the L1 norm of the features for each edge of the graph.
        return aggr_out

In [ ]:
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn.pool import global_max_pool
from torch_geometric.transforms import Cartesian

# Define a simple graph with two nodes and one edge.
edge_index = torch.tensor([[0, 1, 0, 2, 0, 3],
                           [1, 0, 2, 0, 3, 0]], dtype=torch.long)
x = torch.tensor([[1.0], [2.0], [3.0], [4.0]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
edge_attr = torch.tensor([[1.0], [2.0], [3.0], [4.0], [5.0], [6.0]], dtype=torch.float)

# Create an instance of our custom MessagePassing class.
mp = torch_geometric.nn.conv.GCNConv(in_channels=1, out_channels=3)
spConv = torch_geometric.nn.conv.SplineConv(in_channels=1, out_channels=3,dim = 1, kernel_size=2)
sl = SmoothingLayer()

# Perform a forward pass.

x = spConv(x=data.x, edge_index=data.edge_index, edge_attr=edge_attr)
x = mp(x=data.x, edge_index=data.edge_index)
x = sl(x=x, edge_index=data.edge_index)


In [ ]:

# Define the optimizer to use for training.
optimizer = torch.optim.Adam(mp.parameters(), lr=0.001)

weights = []
grads = []
losses = []
ops = []
# Run the message passing algorithm for one iteration.


for i in range(1000):
    
    optimizer.zero_grad()
    x = mp(x=data.x, edge_index=data.edge_index)
    ops.append(x.clone().detach().cpu().numpy()[:,1:2])
    x = sl(x=x, edge_index=data.edge_index)
    x = global_max_pool(x=x, batch=data.batch)
    loss = torch.sum(x)
    losses.append(loss.clone().detach().cpu().numpy())

    # Backpropagate the gradients and update the weights.
    weights.append(mp.bias.clone().detach().cpu().numpy())
    loss.backward()
    grads.append(mp.lin.weight.grad.clone().detach().cpu().numpy())
    optimizer.step()

In [ ]:
import matplotlib.pyplot as plt
#plot each weight in the network
weights_plot = [[w[i] for w in weights] for i in range(weights[0].shape[0])]
grads_plot = [[g[i] for g in grads] for i in range(grads[0].shape[0])]
losses_plot = [l for l in losses]
ops_plot = [[o[i] for o in ops] for i in range(ops[0].shape[0])]

# plot the elemts of grads_plot list in one plot
for i in range(len(ops_plot)):
    plt.plot(ops_plot[i])
plt.show()

# plot the elemts of weights_plot list in one plot
for i in range(len(weights_plot)):
    plt.plot(weights_plot[i])
plt.show()

# plot the elemts of grads_plot list in one plot
for i in range(len(grads_plot)):
    plt.plot(grads_plot[i])
plt.show()

# plot the elemts of losses_plot list in one plot
plt.plot(losses_plot)
plt.show()

In [ ]:
import torch
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.utils import erdos_renyi_graph, remove_isolated_nodes, barabasi_albert_graph, to_undirected

# Define the device to use for training.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Generate some random graph data.
num_graphs = 1000
num_edges = 4
max_nodes = 100
edge_prob = 0.2
num_features = 12

data_list = []


for i in range(num_graphs):
    num_nodes = torch.randint(1, max_nodes, size=(1,))[0]
    # num_nodes = max_nodes
    # Create a random graph with 10 nodes and a 20% probability of an edge between any two nodes,
    # without isolated nodes.
    # edge_index = barabasi_albert_graph(num_nodes = num_nodes, num_edges = num_edges)
    edge_index = erdos_renyi_graph(num_nodes=num_nodes, edge_prob=edge_prob, directed=False)
    # edge_index = to_undirected(edge_index)
    edge_index, _, _ = remove_isolated_nodes(edge_index)
    
    # Create some random node features.
    x = torch.randn(num_nodes, num_features)

    # Create a data object for the graph.
    data = Data(x=x, edge_index=edge_index)
    data_list.append(data)

# Create a data loader with batch size 16.
loader = DataLoader(data_list, batch_size=8, shuffle=True)

In [ ]:
import torch_geometric
from torch_geometric.nn.pool import global_max_pool,global_mean_pool

# Create an instance of our custom MessagePassing class.
mp = torch_geometric.nn.conv.GCNConv(in_channels=num_features, out_channels=2).to(device)
sl = SmoothingLayer()

# Define the optimizer to use for training.
optimizer = torch.optim.Adam(mp.parameters(), lr=0.001)

weights = []
losses = []
grads = []

for epoch in range(25):
    loss_all = 0

    # Iterate over the batches in the data loader.
    for data in loader:
        data = data.to(device)
        # Perform a forward pass.
        x = mp(x=data.x, edge_index=data.edge_index)
        x = sl(x=x, edge_index=data.edge_index)
        x = global_max_pool(x=x, batch=data.batch)
        loss = torch.sum(x)
        optimizer.zero_grad()
        weights.append(mp.lin.weight.clone().detach().cpu().numpy())
        losses.append(loss.item())
        # mp.lin.weight.register_hook(lambda grad: print(grad))
        # Backpropagate the gradients and update the weights.
        loss.backward()
        # print('after:', loss.grad)
        grads.append(mp.lin.weight.grad.clone().detach().cpu().numpy())
        optimizer.step()

        loss_all += loss.item() * data.num_graphs

    # Compute the average loss across all batches.
    loss_all /= len(data_list)

    # Print the average loss for this epoch.
    print('Epoch {}, Loss {:.4f}'.format(epoch, loss_all))
